In [0]:
%run ../utils/common

In [0]:

dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_leadtime_order_vendor
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS

WITH dc_golive AS (
    SELECT DISTINCT dc_id
    FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_dc_golive
),

po_item AS (
    SELECT DISTINCT
        creation_date,
        po_doc,
        store_id,
        ltrim('0',vendor) AS vendor_id,
        gr_date
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_item
    WHERE creation_date BETWEEN ADD_MONTHS(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1, -6)
                          AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1
      AND po_type IN ('Standard PO', 'Collective PO')
      AND DATEDIFF(gr_date, creation_date) >= 2
),

po_percentile AS (
    SELECT 
        store_id,
        vendor_id,
        percentile_approx(DATEDIFF(gr_date, creation_date), 0.25) AS q1_leadtime,
        percentile_approx(DATEDIFF(gr_date, creation_date), 0.5) AS q2_leadtime,
        percentile_approx(DATEDIFF(gr_date, creation_date), 0.75) AS q3_leadtime
    FROM po_item
    GROUP BY store_id, vendor_id
),

min_max_avg AS (
    SELECT
        a.store_id,
        a.vendor_id,
        AVG(DATEDIFF(gr_date, creation_date)) AS avg_leadtime,
        MIN(DATEDIFF(gr_date, creation_date)) AS min_leadtime,
        MAX(DATEDIFF(gr_date, creation_date)) AS max_leadtime,
        COALESCE(STDDEV(DATEDIFF(gr_date, creation_date)), 0) AS stddev_leadtime
    FROM po_item a
    INNER JOIN dc_golive b ON a.store_id = b.dc_id
    GROUP BY a.store_id, a.vendor_id
)

SELECT 
    DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1 AS calday,
    a.store_id AS dc_id,
    a.vendor_id,
    LEAST(10, GREATEST(CEIL(a.avg_leadtime), 2)) AS leadtime,
    a.avg_leadtime,
    a.min_leadtime,
    a.max_leadtime,
    a.stddev_leadtime,
    b.q1_leadtime,
    b.q2_leadtime,
    b.q3_leadtime
FROM min_max_avg a
LEFT JOIN po_percentile b
    ON a.store_id = b.store_id AND a.vendor_id = b.vendor_id
""")
